In [ ]:
# Load packages
if (!require("pacman")) install.packages("pacman")
    pacman::p_load("dplyr", "purrr", "stringr", "httr", "jsonlite", "readr", "here", "lubridate", "tidyr", "taxize", "geosphere", "climwin")
install.packages("jsonlite")
library(jsonlite)

# TODO: Place these keys in a keyring (or something similar)

json_file <- "knmi_keys.json"
data <- fromJSON(json_file)

# Assign values to variables
param_dataverse_api_key = data['param_dataverse_api_key']
param_knmi_edr_api_key = data['param_knmi_edr_api_key']

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error: lexical error: invalid char in json text.
                                       knmi_keys.json
                     (right here) ------^



In [ ]:
# Retrieve bud burst data (as Darwin Core)
#print(param_dataverse_api_key)
# Part I: Retrieve data ---------------------------------------------------

# Function to retrieve data from Dataverse
retrieve_dataverse_data <- function(dataset,
                                    version = ":latest",
                                    server = "demo.dataverse.nl",
                                    key = param_dataverse_api_key) {

  # Check if dataset is provided in right format (i.e., starting with "doi:")
  if(!stringr::str_starts(string = dataset, pattern = "doi:")) {
    dataset_doi <- paste0("doi:", stringr::str_remove(string = dataset, pattern = "DOI:|https://doi.org/"))
  } else {

    dataset_doi <- dataset

  }

  # Retrieve ID that belongs to the data set of interest
  dataset_id <- httr::GET(url = paste0("https://", server, "/api/",
                                       "datasets/:persistentId?persistentId=", dataset_doi),
                          httr::add_headers("X-Dataverse-key" = key)) |>
    httr::content(as = "text", encoding = "UTF-8") |>
    jsonlite::fromJSON() |>
    purrr::pluck("data") |>
    purrr::pluck("id")

  # Retrieve list of data files that are part of the data set
  dataset_files <- httr::GET(url = paste0("https://", server, "/api/",
                                          "datasets/", dataset_id, "/",
                                          "versions/", version, "/",
                                          "files"),
                             httr::add_headers("X-Dataverse-key" = key)) |>
    httr::content(as = "text", encoding = "UTF-8") |>
    jsonlite::fromJSON() |>
    purrr::pluck("data") |>
    purrr::pluck("dataFile")

  # Retrieve each data file in list using their unique IDs
  data <- purrr::map(.x = dataset_files$id,
                     .f = ~{

                       httr::GET(url = paste0("https://", server, "/api/",
                                              "access/datafile/", .x),
                                 httr::add_headers("X-Dataverse-key" = key)) |>
                         httr::content(encoding = "UTF-8")

                     }) |>
    purrr::set_names(stringr::str_remove_all(string = dataset_files$filename, "\\..*"))

  # If API is unsuccessful, prompt message to check DOI, version and/or server
  if(purrr::is_empty(data)) {

    stop("Dataverse API failed to fulfill the request. Check whether the provided dataset DOI, version, and/or server are correct.")

  } else {

    return(data)

  }

}

# Retrieve data
dataverse_list <- retrieve_dataverse_data(dataset = "doi:10.80227/test-QMGPSW")

# Store each table as separate R objects for easier use, and exclude README
purrr::walk2(.x = names(dataverse_list)[-1],
             .y = dataverse_list[-1],
             .f = ~{

               assign(.x, .y, envir = globalenv())

             })

# Part II: create event file (= core of DwC-Archive) ----------------------

## 1. Create help file to create event IDs ####

# Assign area names and abbreviations to sampled trees
d_tree <- tbl_tree %>%
  dplyr::left_join(tbl_area %>%
                     dplyr::select("AreaID", "AreaName", "AreaAbr" = "AreaShortName"),
                   by = "AreaID")

# Assign area names to bud burst data
d_budburst <- tbl_budburst %>%
  dplyr::left_join(d_tree %>%
                     dplyr::select("Area" = "AreaName", "TreeID", "AreaAbr"),
                   by = "TreeID")

# Rename Hoge Veluwe to avoid problems with space between words
d_budburst <- d_budburst %>%
  dplyr::mutate("Area" = stringr::str_replace(string = Area, pattern = " ", replacement = "_"))

# Create eventDate and (hierarchical) eventID of:
# level 1 (<area abbreviation><year>; e.g., HV1988)
# level 2 (<level 1 ID>_<day of year>; e.g., HV1988_119)
# level 3 (<level 2 ID>_<TreeID>; e.g., HV1988_119_412)
hierarchical_events <-
  d_budburst %>%
  dplyr::select("Year", "Month", "Day", "BudburstID", "Area", "AreaAbr", "TreeID") %>%
  dplyr::mutate(eventDate = lubridate::make_date(year = Year, month = Month, day = Day),
                DOY = lubridate::yday(eventDate),
                eventID_L1 = paste(AreaAbr, Year, sep = ""),
                eventID_L2 = paste(eventID_L1, DOY, sep = "_"),
                eventID_L3 = paste(eventID_L2, TreeID, sep = "_"))

## 2. Create event file for Level 1: Years ####

# Get all areas sampled within each year
areas_per_year <-
  d_budburst %>%
  dplyr::group_by(Year) %>%
  dplyr::distinct(Area, .keep_all = TRUE) %>%
  dplyr::summarise(location = paste(Area, collapse = ", ")) %>%
  dplyr::ungroup()

# Create associated event file
d_events_level1 <-
  hierarchical_events %>%
  dplyr::select("eventID_L1", "Year") %>%
  dplyr::distinct(eventID_L1, .keep_all = TRUE) %>%
  dplyr::mutate(eventDate = as.character(Year),
                month = NA,
                day = NA,
                samplingProtocol = NA,
                sampleSizeValue = NA,
                sampleSizeUnit = NA,
                parentEventID = NA,
                decimalLatitude = NA,
                decimalLongitude = NA,
                geodeticDatum = NA,
                minimumElevationInMeters = NA,
                maximumElevationInMeters = NA,
                verbatimLocality = areas_per_year$location[match(.$Year, areas_per_year$Year)]) %>%
  dplyr::rename("eventID" = "eventID_L1",
                "year" = "Year")

## 3. Create event file for level 2: Sampling day within each year ####

# Get all the areas that have been sampled on a specific day (in each year)
areas_per_day <-
  hierarchical_events %>%
  dplyr::group_by(eventDate) %>%
  dplyr::distinct(Area, .keep_all = TRUE) %>%
  dplyr::summarise(location = paste(Area, collapse = ", ")) %>%
  dplyr::ungroup()

# Create event file
d_events_level2 <-
  hierarchical_events %>%
  dplyr::select("eventID_L2", "eventID_L1", "eventDate", "Year", "Month", "Day") %>%
  dplyr::distinct(eventID_L2, .keep_all = TRUE) %>%
  dplyr::mutate(samplingProtocol = NA,
                sampleSizeValue = NA,
                sampleSizeUnit = NA,
                decimalLatitude = NA,
                decimalLongitude = NA,
                geodeticDatum = NA,
                minimumElevationInMeters = NA,
                maximumElevationInMeters = NA,
                verbatimLocality = areas_per_day$location[match(.$eventDate, areas_per_day$eventDate)]) %>%
  dplyr::rename("eventID" = "eventID_L2",
                "parentEventID" = "eventID_L1",
                "year" = "Year",
                "month" = "Month",
                "day" = "Day") %>%
  # Convert dates to characters to avoid merging problems later on
  dplyr::mutate(eventDate = as.character(eventDate))


## 4. Create event file for level 3: bud burst observation on each tree within each year ####

# Connect Tree table to bud burst table and h1
d_budburst <- d_budburst %>%
  dplyr::left_join(hierarchical_events %>%
                     dplyr::select("eventID" = "eventID_L3", "BudburstID"),
                   by = "BudburstID")

# Create associated event file
d_events_level3 <-
  hierarchical_events %>%
  dplyr::select("eventID_L3", "eventID_L2", "eventDate", "Year", "Month", "Day", "TreeID") %>%
  dplyr::mutate(samplingProtocol = "https://doi.org/10.1098/rspb.2000.1363",
                sampleSizeValue = 1,
                sampleSizeUnit = "tree",
                decimalLatitude = d_tree$Latitude[match(.$TreeID, d_tree$TreeID)],
                decimalLongitude = d_tree$Longitude[match(.$TreeID, d_tree$TreeID)],
                minimumElevationInMeters = d_tree$Elevation[match(.$TreeID, d_tree$TreeID)],
                maximumElevationInMeters = minimumElevationInMeters,
                verbatimLocality = d_budburst$Area[match(.$eventID_L3, d_budburst$eventID)]) %>%
  dplyr::rename("eventID" = "eventID_L3",
                "parentEventID" = "eventID_L2",
                "year" = "Year",
                "month" = "Month",
                "day" = "Day") %>%
  dplyr::select(!"TreeID")

# Add geodetic Datum only for events where coordinates are given
d_events_level3 <-
  d_events_level3 %>%
  dplyr::mutate(geodeticDatum = dplyr::case_when(!is.na(decimalLatitude) ~ "EPSG:4326",
                                                 TRUE ~ NA_character_),
                eventDate = as.character(eventDate))

# Combine all three event files into the final event-core file
event <-
  dplyr::bind_rows(d_events_level1, d_events_level2, d_events_level3) %>%
  dplyr::arrange(eventDate, eventID)

# Add DwC columns that apply to all event levels
event <-
  event %>%
  dplyr::mutate(language = "en",
                country = "Netherlands",
                countryCode = "NL",
                institutionID = "https://ror.org/01g25jp36",
                institutionCode = "NIOO",
                type = "Event") %>%
  # Reorder event file according to GBIF list
  dplyr::select("eventID", "parentEventID", "samplingProtocol", "sampleSizeValue",
                "sampleSizeUnit", "eventDate", "year", "month", "day", "country",
                "countryCode", "verbatimLocality", "minimumElevationInMeters",
                "maximumElevationInMeters", "decimalLatitude", "decimalLongitude",
                "geodeticDatum", "type", "language", "institutionID", "institutionCode") %>%
  # Rename "Hoge Veluwe" back to original name
  dplyr::mutate(verbatimLocality = stringr::str_replace(string = verbatimLocality, pattern = "_", replacement = " "))

# Save file as text file
write.csv(event, file = here::here("data", "event.csv"), row.names = FALSE)


# Part III. Create occurrence table ---------------------------------------

# Merge tables to assign tree species to each measurement
tree_species <-
  d_tree %>%
  dplyr::select("TreeID", "TreeSpeciesID") %>%
  #dplyr::select("TreeID", "TreeSpeciesID", "Remarks") %>%
  dplyr::left_join(tbl_treeSpecies, by = "TreeSpeciesID") %>%
  dplyr::right_join(d_budburst, by = "TreeID")


## 1. Get the taxonomic information of all species ####

# Add scientific names to tree table
tree_species <-
  tree_species %>%
  dplyr::mutate(species = dplyr::case_when(TreeSpeciesName == "European oak" ~ "Quercus robur",
                                           TreeSpeciesName == "American oak" ~ "Quercus rubra",
                                           TreeSpeciesName == "Larch" ~ "Larix kaempferi",
                                           TreeSpeciesName == "Pine" ~ "Pinus sylvestris",
                                           TreeSpeciesName == "Birch" ~ "Betula pendula",
                                           TRUE ~ "Tracheophyta"))

# Get all scientific Names to query the taxonomic information in the next step
sciNames <- unique(tree_species$species)

# Query for all species
tax <- taxize::get_gbifid_(sci = sciNames) %>%
  dplyr::bind_rows() %>%
  dplyr::filter(status == "ACCEPTED" & matchtype == "EXACT") %>%
  tidyr::separate(canonicalname, c("Genus", "specificEpithet"), remove = FALSE) %>%
  dplyr::select("scientificName" = "scientificname", "canonicalname",
                "kingdom", "phylum", "class", "order", "family", "genus", "specificEpithet")


# Bind taxonomic information to each observation
tree_species_tax <- dplyr::left_join(tree_species,
                                     tax,
                                     by = c("species" = "canonicalname"))


## 2. Create occurrence IDs ####

# Check whether there is any occasion in which more than one tree was sampled at a sampling event
# (should not be the case here as we know that one measurement is only one tree at a time)
if(d_budburst %>% dplyr::count(eventID) %>% dplyr::filter(n > 1) %>% nrow() > 0) {

  stop(paste("In", d_budburst %>% dplyr::count(eventID) %>% dplyr::filter(n > 1) %>% nrow(),
             "instances of an event, more than one tree was sampled.",
             "This should not be the case for level-3 events."))

}

# Create occurrenceID by extending eventID with '_1'
occID <-
  d_events_level3 %>%
  dplyr::arrange(eventDate) %>%
  dplyr::mutate(occurrenceID = paste(eventID, 1, sep = "_"))

# Create occurrence file
occurrence <-
  tree_species_tax %>%
  dplyr::select("eventID", #"ObserverName",
                "kingdom", "phylum", "class", "order",
                "family", "genus", "specificEpithet", "scientificName", "TreeID") %>%
  dplyr::mutate(individualCount = 1,
                basisOfRecord = "HumanObservation",
                occurrenceStatus = "present",
                occurrenceRemarks = NA,
                recordedBy = NA, ## TODO: Replace when observer names are anonymised
                occurrenceID = occID$occurrenceID[match(.$eventID, occID$eventID)]) %>%
  dplyr::rename(#"recordedBy" = "ObserverName",
                "organismID" = "TreeID") %>%
  dplyr::select("eventID", "occurrenceID", "recordedBy",
                "individualCount", "basisOfRecord", "occurrenceStatus",
                "occurrenceRemarks", "organismID", "scientificName", "kingdom", "phylum", "class", "order",
                "family", "genus", "specificEpithet")

# Save file as text file
write.csv(occurrence, file = here::here("data", "occurrence.csv"), row.names = FALSE)

# Part IV: Create Measurement or fact file --------------------------------

## 1. Create measurement or fact file ####
measurement_or_fact <-
  tree_species_tax %>%
  tidyr::pivot_longer(col = c("TreeTopScore", "TreeAllScore"),
                      names_to = "measurementType",
                      values_to = "measurementValue")  %>%
  dplyr::select("eventID", "measurementType", "measurementValue")%>%
  dplyr::mutate(measurementUnit = NA,
                measurementMethod = "https://doi.org/10.1098/rspb.2000.1363",
                measurementRemarks = NA)

## 2. Create measurementID ####

# Number the measurements per occurrence
measurement_or_fact <-
  measurement_or_fact %>%
  dplyr::group_by(eventID) %>%
  dplyr::mutate("ID" = 1:dplyr::n()) %>%
  dplyr::ungroup()

# Add occurrenceID & create measurementID by extending occurrenceID by number of measurement
measurement_or_fact <-
  measurement_or_fact %>%
  dplyr::left_join(occurrence %>%
                     dplyr::select("occurrenceID", "eventID"),
                   by = "eventID") %>%
  dplyr::mutate(measurementID = paste(occurrenceID, ID, sep = "_")) %>%
  dplyr::select(!c("ID", "occurrenceID")) %>%
  # Rename measurement types to fit more controlled vocabulary
  dplyr::mutate(measurementType = dplyr::case_when(measurementType == "TreeTopScore" ~ "bud burst stage (PO:0025532) of the tree crown",
                                                   measurementType == "TreeAllScore" ~ "bud burst stage (PO:0025532) of the whole tree")) %>%
  # Reorder columns according to GBIF list
  dplyr::select("measurementID", "eventID", "measurementType", "measurementValue",
                "measurementUnit", "measurementMethod", "measurementRemarks")

# Save file as text file
write.csv(measurement_or_fact, file = here::here("data", "extendedmeasurementorfact.csv"), row.names = FALSE)

In [ ]:
# Process bud burst data

# Part I: Preparation  ----------------------------------------------------

# Load DwC-A files
event <- read.csv(here::here("data", "event.csv"))
occ <- read.csv(here::here("data", "occurrence.csv"))
mof <- read.csv(here::here("data", "extendedmeasurementorfact.csv"))

# Connect measurements with events (i.e., trees)
d_bb <- dplyr::left_join(event, mof, by = "eventID")

# Get organism ID for each event
d_bb <-
  occ %>%
  dplyr::select("eventID", "organismID", "scientificName") %>%
  dplyr::right_join(d_bb, by = "eventID", relationship = "many-to-many")

# Part II: Queries --------------------------------------------------------

# 1. Define bud burst criteria ####

# Set measurement type criterion: TopScore (= bud burst stage (PO:0025532) of the tree crown)
crit_measType <- "bud burst stage (PO:0025532) of the tree crown"

# First day where measurement value (of the tree crown) > 1 is bud burst date
crit_measValue <- 1


# 2. Select dates before reaching criterion and when reaching criterion per tree per year ####

# Filter for tree crown, which is used to determine bud burst dates
d_bb_crown <-
  d_bb %>%
  dplyr::filter(measurementType == crit_measType) %>%
  ## Convert date to day of year for easier calculations
  dplyr::mutate(DOY = lubridate::yday(eventDate))

# Bud burst date is the first date where the bud burst stage of the tree crown is scored >= 1
# First, we get the earliest dates (and associated bud burst stage values) per tree per year that matches this criterion
min_bb <- d_bb_crown %>%
  dplyr::filter(measurementValue >= crit_measValue) %>%
  dplyr::summarise(min_DOY_above_criterion = min(DOY),
                   min_value = measurementValue[DOY == min_DOY_above_criterion],
                   .by = c("year", "organismID"))

# Join earliest date and value back into main data frame
d_bb_crown2 <- dplyr::left_join(d_bb_crown,
                                min_bb,
                                by = c("year", "organismID"))

# Second, we also need the latest date (and associated value) at which the bud burst stage was below the criterion
# This is used to interpolate a bud burst date in cases where the tree reached stage 1 in-between two field visits
# In other cases, where stage 1 was observed during a field visit, the date of that visit is used as the bud burst date
max_bb <- d_bb_crown2 %>%
  dplyr::filter(measurementValue < crit_measValue & DOY < min_DOY_above_criterion,
                .by = c("year", "organismID")) %>%
  dplyr::summarise(max_DOY_below_criterion = max(DOY),
                   max_value = measurementValue[DOY == max_DOY_below_criterion],
                   .by = c("year", "organismID"))

# Join earliest date at criterion and latest date below criterion, and their recorded bud burst values
min_max_bb <- dplyr::left_join(min_bb,
                               max_bb,
                               by = c("year", "organismID"))

# 3. Calculate bud burst dates per tree per year ####

# For trees for which a stage of 1 was observed during a visit, use the date of that visit as the bud burst date
match_criterion <- min_max_bb %>%
  dplyr::filter(min_value == crit_measValue) %>%
  dplyr::mutate(bud_burst_date = min_DOY_above_criterion + lubridate::make_date(year, 1, 1) - 1) %>%
  dplyr::rename("bud_burst_DOY" = "min_DOY_above_criterion")

# For trees for which a stage 1 was NOT observed during a visit, (linearly) interpolate the date
# from the latest date before criterion and the earliest date after reaching criterion
interpolated <- min_max_bb %>%
  dplyr::filter(min_value != crit_measValue) %>%
  dplyr::mutate(diff_date = min_DOY_above_criterion - max_DOY_below_criterion,
                diff_value = min_value - max_value,
                value_per_day = diff_value / diff_date,
                days_to_reach_criterion = (crit_measValue - max_value) / value_per_day,
                interpolated_DOY = max_DOY_below_criterion + days_to_reach_criterion,
                interpolated_value  = max_value + days_to_reach_criterion * value_per_day) %>%
  dplyr::mutate(bud_burst_date = round(interpolated_DOY) + lubridate::make_date(year, 1, 1) - 1) %>%
  dplyr::rename("bud_burst_DOY" = "interpolated_DOY")

# Merge determined or calculated bud burst dates and bud burst DOYs per tree per year together
# and add auxiliary info (i.e., locality, scientific name) back in from main file
bud_burst_dates <- dplyr::bind_rows(match_criterion %>%
                                      dplyr::select("year", "organismID", "bud_burst_date", "bud_burst_DOY"),
                                    interpolated %>%
                                      dplyr::select("year", "organismID", "bud_burst_date", "bud_burst_DOY")) %>%
  dplyr::left_join(d_bb_crown %>%
                     dplyr::select("year", "organismID", "verbatimLocality", "scientificName") %>%
                     dplyr::distinct(),
                   by = c("year", "organismID")) %>%
  dplyr::arrange(year, organismID)

# Save dates as csv
# We save both "bud_burst_date" (whole "integer" dates) and "bud_burst_DOY" (allowing non-integers)
write.csv(bud_burst_dates, here::here("data", "annual_budburst_per_tree.csv"), row.names = FALSE)

In [ ]:
# Get temperature data from KNMI

# I. Function to retrieve data from the EDR API ------------------------------

# Arguments:
# bbox: spatial bounding box for which to retrieve data. Vector of four numeric values, indicating western-most, southern-most, eastern-most and northern-most point of the bounding box (in decimal degrees).
# variable: weather variable of interest. Either "mean temperature", "max temperature", "min temperature" or "precipitation".
# start_date: start date of period for which to retrieve data. Date format "yyyy-mm-dd". Use e.g., `lubridate::make_date()`.
# start_time: start time of period for which to retrieve data. Character string of format "hh:mm:ss". Default: "00:00:00".
# end_date: end date of period for which to retrieve data. Date format "yyyy-mm-dd". Use e.g., `lubridate::make_date()`.
# end_time: end time of period for which to retrieve data. Character string of format "hh:mm:ss". Default: "23:59:59".
# knmi_edr_key: user-specific KNMI EDR API key (request here: https://developer.dataplatform.knmi.nl/edr-api#token). Character string.

retrieve_knmi_edr_data <- function(bbox,
                                   variable = c("mean temperature", "max temperature",
                                                "min temperature", "precipitation"),
                                   start_date,
                                   start_time = "00:00:00",
                                   end_date,
                                   end_time = "23:59:59",
                                   knmi_edr_key = param_knmi_edr_api_key) {

  # Create KNMI variable lookup table to match variable inputs to KNMI collection & parameter names
  knmi_var_lookup <- tibble::tibble(
    collection = c("Tg1", "Tx1", "Tn1", "Rd1"),
    parameter = c("temperature", "temperature", "temperature", "precipitation"),
    var_name = c("mean temperature", "max temperature", "min temperature", "precipitation")
  )

  if(!(variable %in% knmi_var_lookup$var_name)) {

    stop("The weather variable you provided does not exist. Select one of: 'mean temperature', 'max temperature', 'min temperature', or 'precipitation'.")

  }

  # Check that start_date and end_date are of class 'Date' to ensure that we can retrieve
  if(!any(class(start_date) == "Date", class(end_date) == "Date")) {

    stop("Please provide dates as 'yyyy-mm-dd'.")

  }

  repeat({ # If retrieving data from KNMI EDR API fails, try again

    # Send GET request to KNMI EDR API
    edr_get <- httr::GET(url = paste0("https://api.dataplatform.knmi.nl/edr/collections/",
                                      knmi_var_lookup |> dplyr::filter(var_name == variable) |> dplyr::pull("collection"),
                                      "/cube?bbox=", paste(bbox, collapse = "%2C"),
                                      "&z=0",
                                      "&parameter-name=",
                                      knmi_var_lookup |> dplyr::filter(var_name == variable) |> dplyr::pull("parameter"),
                                      "&datetime=",
                                      start_date, "T",
                                      stringr::str_replace_all(string = start_time,
                                                               pattern = ":",
                                                               replacement = "%3A"), "Z%2F",
                                      end_date, "T",
                                      stringr::str_replace_all(string = end_time,
                                                               pattern = ":",
                                                               replacement = "%3A"), "Z"),
                         httr::add_headers(Authorization = knmi_edr_key))

    # Convert from JSON to R readable format
    edr_data <- jsonlite::fromJSON(txt = rawToChar(x = edr_get$content))

    # If unsuccessful, print message (and try again)
    if(is.null(edr_data$domain)) message(paste0("KNMI EDR API failed to fulfill request with starting date ",
                                                start_date, ". Will try again."))

    # If successful, end
    if(!is.null(edr_data$domain)) break()

  })

  # Output
  return(edr_data)

}



# II. Get temperature data for desired spatiotemporal parameters ---------

# Define bounding box for Veluwe site
bbox <- c(5.824436777442551, 52.032393019069225, 5.870356194968013, 52.046647934312794)

# Retrieve data for 1988 to 2023
temp <- purrr::map(.x = 1988:2023,
                        .f = ~{

                          # Retrieve data for period 1 (1 Dec to 1 March)
                          period1 <- retrieve_knmi_edr_data(bbox = bbox,
                                                            variable = "mean temperature",
                                                            start_date = lubridate::make_date(.x - 1, 12, 1),
                                                            start_time = "00:00:00",
                                                            end_date = lubridate::make_date(.x, 3, 1),
                                                            end_time = "23:59:59")

                          # Retrieve data for period 2 (2 March to 31 May)
                          period2 <- retrieve_knmi_edr_data(bbox = bbox,
                                                            variable = "mean temperature",
                                                            start_date = lubridate::make_date(.x, 3, 2),
                                                            start_time = "00:00:00",
                                                            end_date = lubridate::make_date(.x, 5, 31),
                                                            end_time = "23:59:59")

                          # Match temperature data to dates and coordinates
                          data1 <- tidyr::expand_grid(date = period1$domain$axes$t$values,
                                                      y = period1$domain$axes$y$values,
                                                      x = period1$domain$axes$x$values) |>
                            dplyr::mutate(temperature = period1$ranges$temperature$values)

                          data2 <- tidyr::expand_grid(date = period2$domain$axes$t$values,
                                                      y = period2$domain$axes$y$values,
                                                      x = period2$domain$axes$x$values) |>
                            dplyr::mutate(temperature = period2$ranges$temperature$values)

                          # Bind output of two periods
                          data <- dplyr::bind_rows(data1, data2)

                          return(data)

                        },
                        .progress = TRUE) |>
  purrr::list_c()


# save temperature data 
write.csv(temp, here::here("data", "Tg1_seasonalTemperature_Dec1987_to_June2023.csv"), row.names = FALSE)

In [ ]:
# Prepare data for climate window analysis

# I. Load packages & retrieve data ----------------------------------------------------------

# Read in DwC-A tables
event <- read.csv(here::here("data", "event.csv"))
occ <- read.csv(here::here("data","occurrence.csv"))
mof <- read.csv(here::here("data","extendedmeasurementorfact.csv"))

#  Read in processed bud burst data
bud_burst_dates <- read.csv(here::here("data","annual_budburst_per_tree.csv"))

# Read in temperature data
temp <- read.csv(here::here("data", "Tg1_seasonalTemperature_Dec1987_to_June2023.csv")) %>%
  dplyr::rename("Longitude" = "x",
                "Latitude" = "y")


# II. Prepare data for the climate models -----------------------------------------

## 1. Get relevant information to map trees to temperature grid cells ####
# The temperature data of the KNMI is gridded and for trees to be modeled with their
# respective temperatures, they first need to be matched to the temperature from the grid cell closest to them.

# Get organismID & scientific name from occurrence file and connect measurements with events (i.e., trees)
budburst <- dplyr::right_join(occ %>%
                                dplyr::select("eventID", "organismID", "scientificName"),
                              event, by = "eventID", relationship = "many-to-many") %>%
  dplyr::right_join(bud_burst_dates, by = c("year", "scientificName", "organismID", "verbatimLocality")) %>%
  # filter for Hoge Veluwe
  dplyr::filter(verbatimLocality == "Hoge Veluwe")

# Get the distinct longitude and latitude for temperature
lon_lat_temp <- temp %>%
  dplyr::distinct(Longitude, Latitude)

# Get all individual trees with location information
trees <- budburst %>%
  dplyr::distinct(organismID, .keep_all = TRUE) %>%
  dplyr::filter(!is.na(decimalLongitude))

# Select coordinates of each individual tree
tree_coords <- trees %>%
  dplyr::select("decimalLongitude", "decimalLatitude")


## 2. Map trees to temperature grid cells ####

# Calculate the geographic distance between each tree & each temperature coordinate pair
distance <- as.data.frame(geosphere::distm(tree_coords, lon_lat_temp))

# Find the minimum distance between tree and temperature coordinates
distance$minPos <- apply(distance, 1, which.min)

# Assign position name to coordinate pairs to match with positions of minimum value
lon_lat_temp$Pos <- 1:nrow(lon_lat_temp)

# Assign coordinates of the temperature measures closest to each tree to bud burst data
budburst1 <- dplyr::left_join(distance, lon_lat_temp, by = c("minPos" = "Pos")) %>%
  dplyr::select("tempLon" = "Longitude",
                "tempLat" = "Latitude") %>%
  dplyr::bind_cols(trees, .) %>%
  dplyr::select("organismID", "tempLon", "tempLat") %>%
  dplyr::right_join(budburst, by = "organismID")


# III. Create location IDs for each temperature measure point ------------------

# Assign locID to each temperature-coordinate pair
# Exclude trees without location information
temp_locations <- budburst1 %>%
  dplyr::distinct(tempLon, tempLat) %>%
  tidyr::drop_na() %>%
  dplyr::mutate(locID = paste0("loc", 1:dplyr::n()))

# Add locID to each single temperature measure
temp <-
  temp %>%
  dplyr::left_join(temp_locations, by = c("Latitude" = "tempLat",
                                          "Longitude" = "tempLon"))

# IV. Annual average bud burst dates  ----------------------------

# Assign locID to bud burst measures and get annual average bud burst date per locID
avg_annual_budburst_dates <-
  budburst1 %>%
  dplyr::left_join(temp_locations, by = c("tempLat", "tempLon")) %>%
  dplyr::summarise(avg_bud_burst_DOY = mean(bud_burst_DOY, na.rm = TRUE),
                   .by = c("locID", "year", "scientificName")) %>%
  dplyr::mutate(avg_bud_burst_date = avg_bud_burst_DOY + lubridate::make_date(year, 1, 1) - 1)

# V. Save output files ----------------------------------------------------

# Save bud burst data
write.csv(avg_annual_budburst_dates, file = here::here("data", "budburst_climwin_input.csv"),
          row.names = FALSE)

# Save temperature data
write.csv(temp, file = here::here("data", "temp_climwin_input.csv"),
          row.names = FALSE)

In [ ]:
# Climate window analysis

# I. Retrieve data & load packages ----------------------------------------

# Retrieve temperature data and bud burst data
temp <- read.csv(here::here("data", "temp_climwin_input.csv"))

avg_annual_budburst_dates <- read.csv(here::here("data", "budburst_climwin_input.csv"))


# II. Date conversion to fit climwin  -------------------------------------

# climwin can only handle dates in the format "dd/mm/yyyy" (as a character), so dates
# have to be converted before modelling. However, there need to be numeric dates for the baseline models too.

# Convert dates of temperature file to climwin format
temp <- temp %>%
  dplyr::mutate(date = lubridate::as_date(date),
                year = lubridate::year(date),
                month = lubridate::month(date),
                day = lubridate::day(date),
                doy = lubridate::yday(date),
                # Create dummy for filtering window later. Format: 312 = March 12, 401 = April 1
                dummy = month * 100 + day,
                factor_date = as.factor(paste(day, month, year, sep = "/")))

# Convert bud burst dates
avg_annual_budburst_dates <- avg_annual_budburst_dates %>%
  dplyr::mutate(date_info = paste(year, floor(avg_bud_burst_DOY)),
                date = strptime(date_info, "%Y %j"),
                date = as.factor(format(as.Date(date), "%d/%m/%Y"))) %>%
  # Create numeric dates to be used in the baseline model &
  # and exclude trees without coordinates
  dplyr::mutate(DOY = lubridate::yday(as.Date(avg_bud_burst_date))) |>
  dplyr::filter(!is.na(date), !is.na(locID))




# III. Climwin: Sliding window model --------------------------------------------

## 1. Function to calculate the window in which bud burst data is best explained by temperature ####

# Arguments
# - biological_data: Tibble specifying the biological input data for the climate model, containing biological dates that are tested in format 'dd/mm/yyyy'. Only necessary when the first window is calculated.
# - reference_day: Numeric vector of 2 values specifying the day and month of the reference day before which climate windows are tested. For example, c(31, 5) for the 31st of March.
# - range: Numeric vector of 2 values specifying the range of days before the reference day in which climate windows are tested. For example, c(181, 0), meaning that windows between 181 days and 0 days before the reference day are tested.
# - window_number: Choice between "first" and "second", specifies whether the first best window should be calculated or a second window based on the first one.
# - first_window: Tibble containing the best model data of the first window/iteration of the function. Used as input data when second window should be calculated.

find_climate_window <- function(biological_data = NULL,
                                range,
                                reference_day,
                                window_number = c("first", "second"),
                                first_window = NULL) {

  # Find 'first' or 'second' climate window
  if(window_number == "first") {

    # Return error if biological data is not provided when searching for first window
    if(is.null(biological_data)) {

      stop("If you want to find a first climate window, provide the biological data as `biological_data`.")

    }

    # Define baseline model
    baseline <- lm(DOY ~ year, data = biological_data)

  } else if(window_number == "second") {

    # Return error if first window output is not provided when searching for second window
    if(is.null(first_window)) {

      stop("If you want to find a second climate window, provide the output of the first iteration of `find_climate_window()` as `first_window`.")

    }

    biological_data <- first_window$biological_data

    # The first window is added as an explanatory variable to the baseline model
    baseline_data <- first_window$best_window[[1]]$BestModelData %>%
      dplyr::rename("first_window" = "climate",
                    "DOY" = "yvar")

    # Define baseline model
    baseline <- lm(DOY ~ year + first_window, data = baseline_data)

  }

  # climwin analysis: Find best window
  best_window <- climwin::slidingwin(baseline = baseline,
                                     xvar = list(Temp = temp$temperature),
                                     cdate = temp$factor_date,
                                     bdate = biological_data$date,
                                     type = "absolute",
                                     refday = reference_day,
                                     spatial = list(biological_data$locID, temp$locID),
                                     range = range,
                                     func = "lin",
                                     stat = "mean")

  # Back calculation of the opening and closing day of the calculated window to calender dates

  # Create a reference year for calculation of start and end date
  # Note: can be any year that is not a leap year, as dates should be calculated on the basis of regular years
  reference_year <- dplyr::if_else(condition = lubridate::leap_year(max(temp$year)),
                                   true = max(temp$year) - 1,
                                   false = max(temp$year))

  # Calculate calender date when window opens
  start_date <- lubridate::make_date(year = reference_year,
                                     month = reference_day[2],
                                     day = reference_day[1]) - best_window$combos[1,]$WindowOpen

  # Calculate calender date when window closes
  end_date <- lubridate::make_date(year = reference_year,
                                   month = reference_day[2],
                                   day = reference_day[1]) - best_window$combos[1,]$WindowClose

  return(tibble::lst(best_window, biological_data, baseline, range, reference_day, start_date, end_date))

}


## 2. Calculate windows for each species of interest ####
first_window_Qrobur <- find_climate_window(biological_data = avg_annual_budburst_dates %>%
                                             dplyr::filter(stringr::str_detect(scientificName, "Quercus robur")) ,
                                           window_number = "first",
                                           reference_day = c(31, 5),
                                           range = c(181, 0))

first_window_Qrubra <- find_climate_window(biological_data = avg_annual_budburst_dates %>%
                                             dplyr::filter(stringr::str_detect(scientificName, "Quercus rubra")),
                                           reference_day = c(31, 5),
                                           range = c(181, 0),
                                           window_number = "first")